# Starting a binary black hole simulation

One of the systems that we are simulating with SpECTRE are binary black hole
(BBH) mergers and their gravitational waves at very high accuracy. In this
tutorial we will start a BBH simulation with SpECTRE and monitor its output.

First, make sure that you have compiled all the executables you need for running
BBH simulations:

```sh
make -j32 cli bbh
```

We will use the SpECTRE command-line interface (CLI) in this session. You have
learned how to access the CLI in the technology check session. Try it now by
exploring the `spectre bbh` subcommand of the CLI:

In [3]:
!spectre bbh --help

Usage: spectre bbh [OPTIONS] COMMAND [ARGS]...

  Pipeline for binary black hole simulations.

Options:
  -h, --help  Show this message and exit.

Commands:
  find-horizon    Find an apparent horizon in volume data.
  generate-id     Generate initial data for a BBH simulation.
  postprocess-id  Postprocess initial data after generation.
  start-inspiral  Schedule an inspiral simulation from initial data.
  start-ringdown  Schedule a ringdown simulation from the inspiral.


These subcommands are part of the BBH pipeline, which you can use to start and
control BBH simulations. Steps in the BBH pipeline are, for example, generating
the initial data, running the inspiral simulation, running the ringdown
simulation, and extracting waveforms.

> Note: all CLI commands are also available in Python, so everything you do with
> the CLI you can also do in Python scripts. We will use Python to work with BBH
> data in the next session.

Using the pipeline, all you need to do to start a BBH simulation is this:

```sh
spectre bbh generate-id -q 1 --chi-A 0 0 0 --chi-B 0 0 0 -D 16 -w 0.015 -a 0 \
  -L 0 -P 8 -d ./01_bbh --evolve
```

That's it! Now your BBH simulation is scheduled on the cluster. Once the job
runs, it will generate initial data, adjust the initial data parameters in a
control loop so the horizon masses and spins match the given input, and then
start an evolution from this initial data.

Of course you can change parameters such as the mass-ratio $q$, the black hole
spins $\mathbf{\chi}_A$ and $\mathbf{\chi}_B$, and the initial orbital
parameters:

- Coordinate separation $D_0$ (`-D`)
- Orbital angular velocity $\Omega_0$ (`-w`)
- Radial expansion velocity $\dot{a}_0$ (`-a`)

Alternatively, you can choose `--eccentricity=0` plus either `--num-orbits`, or
`--time-to-merger`, or one of the above orbital parameters to define the orbit
based on post-Newtonian estimates.

The `-L` and `-P` parameters control the resolution of the initial data.

> To learn more about initial data generation you will have to wait until
> Friday.

## Monitoring your simulation

Now you can monitor your simulation. Try running:

```sh
spectre status
```

It will show you the state of your running simulations on this machine.

> **Pro tip:** Try `spectre status -w 5` in a separate terminal to keep watch of
> your simulations, updating every 5 seconds.

You can also run a dashboard to give you a visual summary of your running
simulations with lots of plots (this is still very experimental):

```sh
python-spectre -m streamlit run $SPECTRE_HOME/tools/Status/Dashboard.py
```

### Example: Plot trajectories

You can use the spectre CLI to inspect the output data of the simulation while
it is running. We will explore this in detail in the next tutorial. For now,
just try to plot the trajectories:

```sh
spectre plot trajectories 01_bbh/002_Inspiral/Segment_*/BbhReductions.h5 \
  -o trajectories.pdf
```

![Trajectories](/home/nilsvu/sxscon-2024-tutorials/images/trajectories.png)

> **Pro tip:** You can also regularly regenerate plots like this to monitor your
> simulation. Try `watch spectre plot trajectories ...`.

## Starting the inspiral or ringdown separately

The steps of the pipeline are connected, e.g. the `--evolve` flag passed to
`spectre bbh generate-id` above will automatically start the inspiral right
after initial data generation. However, you can also run each step in the
pipeline on its own, e.g. to start the inspiral with different parameters or in
another directory:

```sh
spectre bbh start-inspiral 01_bbh/001_InitialData/InitialData.yaml \
  --num-nodes 6 -O 01_bbh/002a_Inspiral
```

You can also start the inspiral from SpEC initial data:

```sh
spectre bbh start-inspiral path/to/ID_Params.perl -O bbh_spec
```

## Troubleshooting a BBH simulation

## Eccentricity reduction

We have basic support for measuring the orbital eccentricity and updating the
initial orbital parameters to reduce the eccentricity. Work on automation and
improvements of this procedure are ongoing, and mostly independent of the actual
simulation code. You can do a quick estimate of the orbital eccentricity with
this:

```sh
spectre eccentricity-control 01_bbh/002_Inspiral/Segment_0000/BbhReductions.h5 \
  --tmin 500 --tmax 2000 -o eccentricity.pdf
```